In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import os
import sys
import os.path as op

sys.path.insert(0, '..') # project folder
path_data = op.join('..', 'data', 'raw')
path_metadata = op.join(path_data, "yt_metadata_en.jsonl.gz")
path_channels = op.join(path_data, "df_channels_en.tsv.gz")
path_deriv = op.join(path_data, '..', 'derivatives')
path_edu = op.join(path_deriv, "Education_videos_{}.csv")
path_edu_clean = op.join(path_deriv, "Education_videos_{}clean.csv")

In [8]:
df_channels = pd.read_csv(path_channels, compression="infer", sep="\t")

## Channel data 

In [ ]:
df_channels = pd.read_csv(path_channels, compression="infer", sep="\t")
df_channels["join_date"] = pd.to_datetime(df_channels["join_date"])

In [5]:
df_edu = df_channels.loc[df_channels['category_cc'].isin(['Education'])]
df_edu

,category_cc,join_date,channel,name_cc,subscribers_cc,videos_cc,subscriber_rank_sb,weights
1,Education,2006-09-01,UCbCmjCuTUZos6Inko4u57UQ,Cocomelon - Nursery ...,60100000,458,7.0,2.0870
22,Education,2013-02-09,UCBnZ16ahKA2DZ_T5W0FPUXg,ChuChu TV Nursery Rh...,26600000,317,44.0,2.0870
27,Education,2011-12-14,UCcdwLMPsaU2ezNSJU1nFoBQ,Pinkfong! Kids' Song...,23200000,1382,55.0,2.0870
39,Education,2013-10-25,UCRx3mKNUdl8QE06nEug7p6Q,Billion Surprise Toy...,24700000,150,83.0,2.0870
46,Education,2011-06-22,UCKAqou7V9FAWXpZd9xtOg3Q,Little Baby Bum - Nu...,19900000,958,93.0,2.0870
...,...,...,...,...,...,...,...,...
136282,Education,2012-11-21,UCccoliC479T88ziS1TxeLeg,skunkpaste,10074,146,977576.0,53.1435
136291,Education,2013-09-05,UCu13MNNOLhyRV-YtlYWV4tw,Matthew Williams,10000,374,977915.0,53.1435
136316,Education,2015-08-12,UCRoZf8DdJzF5Xi99gtFDY0A,Geoid,10276,136,979269.0,53.1435
136332,Education,2013-09-10,UCax581uo5yNzdgODBdH67cw,Bond Robin,10225,469,980820.0,53.1435


In [ ]:
# faire des sous catégories dans education, 
# plot : Camembert avec les differentes sous categories, rayon est le nombre total 
# how to - match channels with their videos and try to get the theme of them by using the description, title and tags fiels

## Extraction youtube metadata (legacy - see code Viva)

In [ ]:
import random
import json
import gzip

def filter_jsonl(input, category, batch_size, random_seed, all = False):
    filtered_data = []
    random.seed(random_seed)
    counter = 0
    with gzip.open(input, 'rt', encoding='utf-8') as f:
        for line in f:
            entry = json.loads(line)

            if entry.get('categories') == category:
                counter +=1
                if len(filtered_data) <= batch_size or all:
                    filtered_data.append(entry)
                else:
                    index_to_replace = random.randint(0, len(filtered_data) - 1)
                    if index_to_replace < batch_size:
                        filtered_data[index_to_replace] = entry
      
                
    print(f"There are {counter} videos in the Education category!")
    return pd.DataFrame(filtered_data)

            

df = filter_jsonl(path_metadata, 'Education', 500000, 0, all = False)

df.to_csv(op.join(path_deriv, 'df_edu_all.csv'))

There are 3795564 videos in the Education category!


## Preprocessing of video descriptions

In [ ]:
import re
i = 0
df = pd.read_csv(op.join(path_deriv, path_edu.format(i)), index_col=0) #pd.read_csv(op.join(path_deriv, 'df_edu_all.csv'), index_col=0)
df['desc_clean'] = df['description'].fillna('')
df['text'] = df['title'].fillna('') + " " + df['tags'].fillna('') 

#sort the description separately since way more unrelated text, some titles contain keywords we remove in descriptions

##### debug

In [ ]:
pd.set_option('display.max_colwidth', 80)
df.sample(5)[['desc_clean', 'text']]
# some examples of problematic data
# 32379 crypto video with lots of links
# 35962 marseillaise
# 22678 41830 non alpha and indian
# 34547 numbers accolated
# 17337 very long strings in crypto vids
# 3235 hindu

,desc_clean,text
34620,"A video, that describes, who, I believe, Jesus, is, to me.","Who Jesus Is To Me (Automated Voice) God’sTruthBeingTold,who Jesus is to m..."
35217,"Tim Crane gives a general overview of analytic philosophy and its nature, fo...","Analytic Philosophy: Its History, Science, & Logic Philosophy,Analytic Philo..."
35816,"Get Free Korean Lessons on your Android, iPhone, iPad or Kindle Fire! Click ...","Learn the Top 25 Korean Adjectives Korean Language (Human Language),learn Ko..."
35291,Please thank Lucky Gunner for bringing us today’s video of VangComp Makes Ex...,"VangComp Makes Excellent Shotguns asp,cover your asp,active self protection,..."
10905,Reviewing the EQ 73 and EQ 81 plugins from IK Multimedia.,IK Multimedia EQ 73 & EQ 81 Review - Audio Plugin IK Multimedia (Business Op...


In [ ]:
#df[df['title'].str.contains('twitter')]['title']

"""string = df['description'].iloc[5046].lower()
print(string)
remove_urls(string)#.replace('website', 'website blab bla truc muche promotion')"""

#### pipeline

In [6]:
# everything to lower case so that same words are treated the same
df['text'] = df['text'].apply(lambda x: x.lower() if isinstance(x,str) else x)
df['desc_clean'] = df['desc_clean'].apply(lambda x: x.lower() if isinstance(x,str) else x)

In [7]:
# remove urls links and their associated text
url_pattern = re.compile(r'(https?://\S+|www\.\S+)') 
com_pattern = re.compile(r'([^\s]+\.com)')
fb_pattern = re.compile(r'(facebook\s+page|facebook\s+group)[^\w\s]*.*?(\n|$)')
#lines that start with brands or websites 
link_pattern = re.compile(r'(\n+)(facebook|twitter|pinterest|tumblr|instagram|website|amazon)[^\w\s]*\s+\S+')
link_pattern2 = re.compile(r'(\n+)(facebook|twitter|pinterest|tumblr|instagram|website|amazon)')

#TODO lines that contain these websites in the middle of text - hard to implement since not sure if discard

long_words = re.compile(r'\b[a-zA-Z0-9]{21,}\b') # most words in english are below 20 letters, bigger than that is a crypto wallet id

def remove_urls(text, desc = False):
        text = url_pattern.sub('', text)
        text = com_pattern.sub('', text)
        text = long_words.sub('', text)
        if desc: # titles and tags should keep brand names since these might be the focus of the video
                text = fb_pattern.sub('', text)
                text = link_pattern.sub('', text)
                return link_pattern2.sub('', text)
        return text

df['desc_clean'] = df['desc_clean'].apply(remove_urls, desc = True)
df['text'] = df['text'].apply(remove_urls)
#test = 'https://www.youtube.com/everydaytacticalvids\n my twitter account - https://twitter.com/everydaytactic1\n my facebook group tha tha \n ow'
#remove_urls(test, True)

In [8]:
def clean_non_word(text): #punctuation, underscores seem to evade this regex so add it
        return  re.sub(r'[^\w\s]|_+', ' ', text)

def clean_non_ascii(text): # indian symbols that might still be left
        return  re.sub(r'[^\x00-\x7F]+', ' ', text)

df['desc_clean'] = df['desc_clean'].apply(clean_non_word).apply(clean_non_ascii)
df['text'] = df['text'].apply(clean_non_word).apply(clean_non_ascii)

In [9]:
def clean_numeric(text):
    return re.sub(r'\d+(?![a-zA-Z])', '', text) # numbers that are not accolated to strings : # TODO MAybe all numbers
df['desc_clean'] = df['desc_clean'].apply(clean_numeric)
df['text'] = df['text'].apply(clean_numeric)

In [10]:
def clean_space_newline(text):
    return re.sub(r'\s{2,}', ' ', text.replace('\n', ' ')).strip()

df['desc_clean'] = df['desc_clean'].apply(clean_space_newline)
df['text'] = df['text'].apply(clean_space_newline)

In [ ]:
# Recombine both 
df['text_clean'] = df['text'] + " " + df['desc_clean']
df = df.drop(['text', 'desc_clean'], axis = 1)
df.to_csv(op.join(path_deriv, path_edu_clean.format(i)))

In [ ]:
#optional : make the dataframe smaller 
df = df.drop(['description'], axis = 1)

# Categorize subtopics - BERT

In [ ]:
# no need to perform lemmatization, stemming or stopword removal since BERT handles it 
#df= pd.read_csv(op.join(path_deriv, 'df_edu_500k_clean.csv'))
df= pd.read_csv(op.join(path_deriv, path_edu_clean.format(i)))

In [ ]:
trial = df.copy().sample(50000)
pd.set_option('display.max_colwidth', 80)
trial = trial.reset_index(drop=True)
trial.head(5)

In [ ]:
# Try to extract sub topics with sentence transformers like BERT 
from sentence_transformers import SentenceTransformer # https://sbert.net/
from sklearn.cluster import KMeans

print('1. Encoding model...')
model = SentenceTransformer('all-MiniLM-L6-v2') 
embeddings = model.encode(trial.get('text_clean'))
np.save(op.join(path_deriv, 'embeddings.npy'), embeddings) # 41 mins 

1. Encoding model...


In [20]:
print('2. Starting kmeans...')
num_clusters = 50
kmeans = KMeans(n_clusters=num_clusters, random_state=0)

2. Starting kmeans...


In [ ]:
print('3. Fitting k means...') # 10s
clusters = kmeans.fit(embeddings)
cluster_assignment = clusters.labels_
print('4. Predicting labels...')
trial['cluster'] = clusters.predict(embeddings)

3. Fitting k means...


  File "c:\Users\gbrag\miniconda3\envs\ada\Lib\site-packages\joblib\externals\loky\backend\context.py", line 282, in _count_physical_cores
    raise ValueError(f"found {cpu_count_physical} physical cores < 1")


In [ ]:
for cluster_num in range(num_clusters):
    print(f"Cluster {cluster_num}")
    cluster_data = trial[trial['cluster'] == cluster_num]
    display(cluster_data.sample(5)[['title', 'text_clean', 'tags']])

In [ ]:
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import TfidfVectorizer

cluster_dict = {}

for cluster in range(kmeans.n_clusters): #
    texts = [row['text_clean'] for _, row in trial.iterrows() if row['cluster'] == cluster]

    tfidf = TfidfVectorizer(stop_words='english')
    tfidf_matrix = tfidf.fit_transform(texts)

    lda = LatentDirichletAllocation(n_components=1, random_state=1) # only one category per cluster
    lda.fit(tfidf_matrix)
    
    feature_names = tfidf.get_feature_names_out()
    top_idx = lda.components_[0].argsort()[-3:][::-1]
    
    terms = [feature_names[i] for i in top_idx]
    cluster_dict[cluster] = '.'.join(terms)
    print(f"Cluster {cluster}, {len(texts)} videos: {'.'.join(terms)}")

Cluster 0, 121 videos: asl.sign.language
Cluster 1, 1601 videos: ca.gk.learning
Cluster 2, 1833 videos: trp.tv.live
Cluster 3, 1137 videos: video.youtube.use
Cluster 4, 451 videos: sunday.adelaja.pastor
Cluster 5, 867 videos: tedx.ted.organized
Cluster 6, 1231 videos: money.marketing.online
Cluster 7, 1199 videos: maths.examsolutions.math
Cluster 8, 659 videos: food.bajias.cooking
Cluster 9, 1161 videos: garden.house.gardening
Cluster 10, 1388 videos: kids.cbeebies.learn
Cluster 11, 956 videos: rhymes.songs.nursery
Cluster 12, 2153 videos: excel.data.video
Cluster 13, 1403 videos: english.learn.japanese
Cluster 14, 1263 videos: dr.health.medical
Cluster 15, 975 videos: life.health.mental
Cluster 16, 1331 videos: news.video.use
Cluster 17, 984 videos: business.management.iese
Cluster 18, 1145 videos: chess.history.course
Cluster 19, 1620 videos: tips.remedies.awesome
Cluster 20, 287 videos: san.diego.air
Cluster 21, 1004 videos: survival.asp.knife
Cluster 22, 841 videos: trading.day.mar

In [ ]:
trial['cluster_name'] = trial['cluster'].map(cluster_dict)
trial.to_csv(op.join(path_deriv, 'trial_clustered_50k.csv'))

In [ ]:
unique, counts = np.unique(kmeans.labels_, return_counts=True)
counts

array([ 121, 1601, 1833, 1137,  451,  867, 1231, 1199,  659, 1161, 1388,
        956, 2153, 1403, 1263,  975, 1331,  984, 1145, 1620,  287, 1004,
        841, 1019, 1265,  981, 1286, 1125,  993,  626,  412,  843, 1465,
        109,  872, 1116,  420,  467, 1737, 1382, 1186, 1257,  545,  756,
        599, 1215,  628,  324,  890,  872], dtype=int64)